In [6]:
from test_mongo import MongoExplorer
mongo = MongoExplorer()

In [11]:
import pandas as pd
def calc_gain_long(hour, buy_threshold, sell_threshold):
    df = pd.read_csv(f"tradestation_data/{symbol}_{hour}.csv")
    df = df[(df["VolumeSum"] >= buy_threshold) | (df["VolumeSum"] <= sell_threshold) | df["EoH"]].reset_index(drop=True)
    df["Long"] = (df["VolumeSum"] >= buy_threshold) & (~df["EoH"])
    df["Short"] = (df["VolumeSum"] <= sell_threshold) | (df["EoH"])
    df["Buy Action"] = df["Long"] & (df["Short"].shift() | df["EoH"].shift())
    if len(df) < 1:
        return 0, None
    if df.iloc[0, df.columns.get_loc("Long")]:
        df.iloc[0, df.columns.get_loc("Buy Action")] = True
    df["Sell Action"] = df["Short"] & df["Long"].shift() & (~df["EoH"]).shift()
    df.loc[df["Buy Action"], "Gain"] = -pd.to_numeric(df.loc[df["Buy Action"], "BUY"]) 
    df.loc[df["Sell Action"], "Gain"] = pd.to_numeric(df.loc[df["Sell Action"], "SELL"])
    df = df.round(3)
    return df["Gain"].sum(), df

def calc_gain_short(hour, buy_to_cover_threshold, sell_short_threshold):
    df = pd.read_csv(f"tradestation_data/{symbol}_{hour}.csv")
    df = df[(df["VolumeSum"] >= buy_to_cover_threshold) | (df["VolumeSum"] <= sell_short_threshold) | df["EoH"]].reset_index(drop=True)
    df["Long"] = (df["VolumeSum"] >= buy_to_cover_threshold) | (df["EoH"])
    df["Short"] = (df["VolumeSum"] <= sell_short_threshold) & (~df["EoH"])
    df["Sell Action"] = df["Short"] & (df["Long"].shift() | df["EoH"].shift())
    if len(df) < 1:
        return 0, None
    if df.iloc[0, df.columns.get_loc("Short")]:
        df.iloc[0, df.columns.get_loc("Sell Action")] = True
    df["Buy Action"] = df["Long"] & df["Short"].shift() & (~df["EoH"]).shift()
    df.loc[df["Buy Action"], "Gain"] = -pd.to_numeric(df.loc[df["Buy Action"], "BUY"]) 
    df.loc[df["Sell Action"], "Gain"] = pd.to_numeric(df.loc[df["Sell Action"], "SELL"])
    df = df.round(3)
    return df["Gain"].sum(), df

In [24]:
import numpy as np
import pandas as pd
def find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    df_arr = np.empty((0, 7))
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:00:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df["Dif"] = (df["Price"] - df["Last"])
        df = df.rename(columns={"CurTime": "Time"})
        query_3 = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
        df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query_3)))
        df = pd.merge(df, df_3, on="Time")
        df = df[["Time", "Side", "TotalSize", "Dif", "BUY", "SELL"]]
        df = df[df["BUY"].notna() & df["SELL"].notna()].reset_index(drop=True)
        df["EoH"] = df["Time"] == f'{date}T{hour}:59:50Z'
        df_arr = np.concatenate((df_arr, df.to_numpy()), axis=0)

    df = pd.DataFrame(df_arr, columns=["Time", "Side", "TotalSize", "Dif", "BUY", "SELL", "EoH"])
    df = df.loc[((df["Side"] == "Bid") & (df["Dif"] > 0))
                | ((df["Side"] == "Ask") & (df["Dif"] < 0))
                | df["EoH"]]
    df["VolumeSum"] = df["TotalSize"] * df["Dif"]
    df = df[["Time", "VolumeSum", "BUY", "SELL", "EoH"]]
    df = df.groupby("Time").agg({"VolumeSum": "sum", "BUY": "first", "SELL": "first", "EoH": "first"})
    df.to_csv(f"tradestation_data/{symbol}_{hour}.csv")
    long_arr = np.zeros((len(buy_thresholds) * len(sell_thresholds), 3))
    long_arr[:, 0] = np.repeat(buy_thresholds, len(sell_thresholds))
    long_arr[:, 1] = np.tile(sell_thresholds, len(buy_thresholds))
    short_arr = np.zeros((len(buy_thresholds) * len(sell_thresholds), 3))
    short_arr[:, 0] = np.repeat(buy_thresholds, len(sell_thresholds))
    short_arr[:, 1] = np.tile(sell_thresholds, len(buy_thresholds))
    for i in range(len(long_arr)):
        buy_threshold, sell_threshold = long_arr[i, 0], long_arr[i, 1]
        long_arr[i, -1], _ = calc_gain_long(hour, buy_threshold, sell_threshold)
        short_arr[i, -1], _ = calc_gain_short(hour, buy_threshold, sell_threshold)
    long_df = pd.DataFrame(long_arr, columns = ["Buy Threshold", "Sell Threshold", "Sum"])
    long_df = long_df.round(3)
    long_df.to_csv(f"tradestation_data/{symbol}_{hour}_thresholds_long.csv", index=False, date_format="%Y-%m-%d")
    long_df = long_df.sort_values(by=["Sum"], ascending=False)
    short_df = pd.DataFrame(short_arr, columns = ["Buy Threshold", "Sell Threshold", "Sum"])
    short_df = short_df.round(3)
    short_df.to_csv(f"tradestation_data/{symbol}_{hour}_thresholds_short.csv", index=False, date_format="%Y-%m-%d")
    short_df = short_df.sort_values(by=["Sum"], ascending=False)
    return long_df.iloc[0].tolist(), short_df.iloc[0].tolist()

# symbol = "TSLA"
# buy_thresholds = (np.arange(500))
# sell_thresholds = (-np.arange(500))
# hour = 13
# days = ["13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
# dates = [f"2023-06-{day}" for day in days]
# days = ["03", "05", "06", "07", "10", "11", "12", "13", "14", "17", "18"]
# dates = dates + [f"2023-07-{day}" for day in days]
# print(f"{hour}: {find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds)}")
# days = ["08", "09", "12", "13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
# dates = [f"2023-06-{day}" for day in days]
# days = ["03", "05", "06", "07", "10", "11", "12", "13", "14", "17", "18"]
# dates = dates + [f"2023-07-{day}" for day in days]
# for hour in range(14, 20):
#     print(f"{hour}: {find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds)}")
symbols = ['NVDA', 'AMZN', 'AAPL', 'GOOG']
for symbol in symbols:
    buy_thresholds = (np.arange(500))
    sell_thresholds = (-np.arange(500))
    hour = 13
    days = ["24"]
    dates = [f"2023-07-{day}" for day in days]
    print(f"{hour}: {find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds)}")
    for hour in range(14, 20):
        print(f"{hour}: {find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds)}")


13: ([250.0, 0.0, 0.0], [14.0, -20.0, 5.47])
14: ([1.0, -5.0, 3.75], [2.0, -4.0, 0.95])
15: ([17.0, -182.0, 2.88], [499.0, -499.0, 0.0])
16: ([12.0, -4.0, 1.24], [13.0, -5.0, 4.69])
17: ([29.0, -181.0, 3.46], [499.0, -499.0, 0.0])
18: ([24.0, -32.0, 0.61], [24.0, -1.0, 2.46])
19: ([10.0, -394.0, 1.68], [8.0, -21.0, 1.98])
13: ([250.0, 0.0, 0.0], [250.0, 0.0, 1.7])
14: ([53.0, -50.0, 0.96], [53.0, -52.0, 0.6])
15: ([19.0, -31.0, 0.54], [67.0, -31.0, 0.39])
16: ([67.0, 0.0, 0.07], [42.0, -21.0, 0.84])
17: ([12.0, -346.0, 0.3], [40.0, -56.0, 0.08])
18: ([1.0, -2.0, 0.11], [249.0, -34.0, 0.29])
19: ([43.0, -7.0, 0.13], [37.0, 0.0, 0.34])
13: ([16.0, -42.0, 1.12], [186.0, -46.0, 1.15])
14: ([1.0, -27.0, 0.23], [0.0, -24.0, 0.14])
15: ([15.0, -5.0, 0.32], [400.0, -22.0, 0.45])
16: ([15.0, -2.0, 0.25], [19.0, -2.0, 0.33])
17: ([13.0, -9.0, 0.2], [24.0, -11.0, 0.22])
18: ([5.0, -13.0, 0.28], [7.0, -21.0, 0.66])
19: ([40.0, -493.0, 0.24], [24.0, -6.0, 0.3])
13: ([20.0, -14.0, 1.15], [23.0, -16.

In [5]:
import numpy as np
import pandas as pd
def test_thresholds(symbol, dates, hour, buy_threshold, sell_threshold, buy_to_cover_threshold, sell_short_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    long_gains = []
    short_gains = []
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:00:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df["Dif"] = (df["Price"] - df["Last"])
        df = df.rename(columns={"CurTime": "Time"})
        query_3 = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
        df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query_3)))
        df = pd.merge(df, df_3, on="Time")
        df = df[["Time", "Side", "TotalSize", "Dif", "BUY", "SELL"]]
        df = df[df["BUY"].notna() & df["SELL"].notna()].reset_index(drop=True)
        df["EoH"] = df["Time"] == f'{date}T{hour}:59:50Z'
        #df_arr = np.concatenate((df_arr, df.to_numpy()), axis=0)

        # df = pd.DataFrame(df_arr, columns=["Time", "Side", "TotalSize", "Dif", "BUY", "SELL", "EoH"])
        df = df.loc[((df["Side"] == "Bid") & (df["Dif"] > 0))
                    | ((df["Side"] == "Ask") & (df["Dif"] < 0))
                    | df["EoH"]]
        df["VolumeSum"] = df["TotalSize"] * df["Dif"]
        df = df[["Time", "VolumeSum", "BUY", "SELL", "EoH"]]
        df = df.groupby("Time").agg({"VolumeSum": "sum", "BUY": "first", "SELL": "first", "EoH": "first"})
        df.to_csv(f"tradestation_data/{symbol}_{hour}.csv")
        long_gain, long_df = calc_gain_long(hour, buy_threshold, sell_threshold)
        short_gain, short_df = calc_gain_short(hour, buy_to_cover_threshold, sell_short_threshold)
        # long_df.to_csv(f"tradestation_data/TSLA_{hour}_test_long.csv")
        # short_df.to_csv(f"tradestation_data/TSLA_{hour}_test_short.csv")
        long_gains.append(round(long_gain, 3))
        short_gains.append(round(short_gain, 3))
    return long_gains, sum(long_gains), short_gains, sum(short_gains)
    
symbol = "TSLA"
buy_thresholds = {13: 4.0, 14: 78.0, 15: 63.0, 16: 52.0, 17: 20.0, 18: 9.0, 19: 42.0}
sell_thresholds =  {13: -218.0, 14: -43.0, 15: -216.0, 16: -442.0, 17: -47.0, 18: -70.0, 19: -408.0}
buy_to_cover_thresholds = {13: 25.0, 14: 251.0, 15: 0.0, 16: 199.0, 17: 40.0, 18: 38.0, 19: 110.0}
sell_short_thresholds = {13: -53.0, 14: -8.0, 15: -37.0, 16: -68.0, 17: -39.0, 18: -70.0, 19: -1.0}
hour = 13
# days = ["13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
# dates = [f"2023-06-{day}" for day in days]
days = ["03", "05", "06", "07", "10", "11", "12", "13", "14", "17", "18", "19", "20"]
#dates = dates + [f"2023-07-{day}" for day in days]
dates = [f"2023-07-{day}" for day in days]
print(test_thresholds(symbol, dates, hour, buy_thresholds[hour], sell_thresholds[hour], buy_to_cover_thresholds[hour], sell_short_thresholds[hour]))
# days = ["08", "09", "12", "13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
# dates = [f"2023-06-{day}" for day in days]
days = ["03", "05", "06", "07", "10", "11", "12", "13", "14", "17", "18", "19", "20"]
#dates = dates + [f"2023-07-{day}" for day in days]
dates = [f"2023-07-{day}" for day in days]
for hour in range(14, 20):
    print(test_thresholds(symbol, dates, hour, buy_thresholds[hour], sell_thresholds[hour], buy_to_cover_thresholds[hour], sell_short_thresholds[hour]))


([4.8, 1.02, -4.3, 0.74, -5.6, 1.27, -2.09, -0.65, 2.86, 3.93, 0.27, 0.9, -2.78], 0.37000000000000144, [-1.47, -2.47, 4.02, -0.44, 4.91, -0.32, 0.66, -0.26, -0.69, -0.36, 1.27, 0.0, 1.5], 6.35)
([-0.58, -0.71, 0.25, 0.14, -0.82, 0.0, 0.0, 0.31, 0.0, -0.22, 1.39, 0.0, 0.0], -0.24, [2.29, 0.51, -1.12, 1.56, 0.04, 0.23, 1.35, 0.72, 1.94, 4.85, 1.51, -1.09, 4.45], 17.24)
([0.0, 0.0, 0.0, 0.0, 0.0, 1.04, 0.0, 0.0, 0.0, 3.67, 0.0, 0.0, 0.54], 5.25, [1.42, 0.4, -0.08, 0.09, -0.14, -0.06, -0.64, 0.26, 0.09, -0.04, 0.0, -0.03, 0.03], 1.3)
([4.0, 0.0, 0.02, 0.0, 0.0, 0.0, 0.0, 0.55, 1.54, 0.62, 0.0, 0.0, -5.46], 1.2699999999999996, [-2.32, -0.34, 0.14, 0.28, 0.0, -0.55, 0.0, 0.0, -1.59, 0.0, 0.0, 2.45, 0.0], -1.9299999999999988)
([-0.46, -0.38, 1.59, 0.43, -0.17, 0.02, 0.77, 0.48, -0.21, 0.45, -0.43, -2.66, 1.85], 1.2800000000000002, [0.97, 0.0, 1.71, 0.43, -0.15, 0.41, 0.0, -1.44, 1.98, -0.34, -0.01, -0.82, 0.2], 2.940000000000001)
([0.17, -0.64, 0.21, -0.04, 0.78, 0.55, 0.0, 2.93, -2.62, 2.05,